In [1]:
!pip install deepface

In [1]:
!pip install transformers torch
!pip install Pillow
!pip install PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [ ]:
import cv2
import time
import numpy as np
from PIL import Image # Transformers ke liye zaroori

# --- Installation ---
# 1. pip install transformers
# 2. pip install torch (ya tensorflow)
# 3. pip install Pillow
try:
    from transformers import pipeline
except ImportError:
    print("Error: 'transformers' library not found.")
    print("Please install it using: pip install transformers torch Pillow")
    exit()

# --- Parameters ---
ANALYSIS_INTERVAL = 1.0  # Har 1 second mein analyze karo (Performance ke liye zaroori)
# Hugging Face se pre-trained model load kar rahe hain
# 'pipeline' function sabse aasan tareeka hai
print("Loading Hugging Face model... (Yeh pehli baar time lega)")
try:
    # --- FIX ---
    # Model: 'jonathano/face-emotion-recognition-vit' (jo fail ho raha tha)
    # Naya, working model: 'trpakov/vit-face-expression'
    emotion_classifier = pipeline("image-classification", 
                                  model="trpakov/vit-face-expression")
    print("Hugging Face model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please check your internet connection or model name.")
    exit() # --- FIX --- Program ko yahi rok dete hain agar model load nahi hua

# Face detection ke liye OpenCV ka model (taaki sirf chehra hi pipeline ko bhejein)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# --- State Variables ---
last_analysis_time = 0
dominant_emotion = "---"
emotion_scores = {} # {'sad': 0.87, 'happy': 0.09, ...}
face_region = None  # (x, y, w, h)

print("Webcam starting, Press 'q' to quit.")
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to open the Webcam.")
    exit()

def draw_dashboard(frame, scores_dict, w, h):
    """
    Screen ke right side par emotion scores (probabilities) ka dashboard banata hai.
    """
    dashboard_width = 300
    overlay = frame.copy()
    cv2.rectangle(overlay, (w - dashboard_width, 0), (w, h), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame) # Transparency

    cv2.putText(frame, "Emotion Analysis (HF)", (w - dashboard_width + 20, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    y_pos = 80
    if not scores_dict:
        cv2.putText(frame, "Detecting...", (w - dashboard_width + 20, y_pos), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        return

    # Scores ko sort karke display karte hain
    sorted_emotions = sorted(scores_dict.items(), key=lambda item: item[1], reverse=True)
    
    for emotion, score in sorted_emotions:
        score_percentage = score * 100
        score_text = f"{emotion.capitalize()}: {score_percentage:.1f}%"
        cv2.putText(frame, score_text, (w - dashboard_width + 20, y_pos), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        # Confidence bar
        bar_width = int(score * (dashboard_width - 60))
        bar_color = (0, 255, 0) # Green
        if emotion == sorted_emotions[0][0]: # Dominant emotion
            bar_color = (0, 0, 255) # Red
        
        cv2.rectangle(frame, (w - dashboard_width + 20, y_pos + 10), 
                      (w - dashboard_width + 20 + bar_width, y_pos + 20), 
                      bar_color, -1)
        
        y_pos += 40

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (1280, 720)) # Resize for dashboard
    frame_h, frame_w, _ = frame.shape
    current_time = time.time()
    
    # --- Analysis Timer ---
    if current_time - last_analysis_time > ANALYSIS_INTERVAL:
        last_analysis_time = current_time
        
        # 1. Pehle OpenCV se chehra detect karte hain
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        
        if len(faces) > 0:
            x, y, w, h = faces[0] # Pehla chehra lete hain
            face_region = (x, y, w, h)
            
            # 2. Chehre ko crop karte hain
            face_crop = frame[y:y+h, x:x+w]
            
            # 3. OpenCV (BGR) ko PIL (RGB) mein convert karte hain
            pil_image = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
            
            # 4. Hugging Face Pipeline se analyze karte hain
            try:
                results = emotion_classifier(pil_image)
                # Results ko dictionary mein badalte hain:
                # [{'label': 'sad', 'score': 0.87}, ...] -> {'sad': 0.87, ...}
                emotion_scores = {item['label']: item['score'] for item in results}
                dominant_emotion = results[0]['label'].capitalize()
            except Exception as e:
                print(f"Error during analysis: {e}")
                dominant_emotion = "Error"
                emotion_scores = {}
        else:
            # Agar chehra nahi mila
            dominant_emotion = "---"
            emotion_scores = {}
            face_region = None

    # --- Frame par Draw karna ---
    
    if face_region:
        x, y, w, h = face_region
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"Emotion: {dominant_emotion}", (x, y - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    else:
        cv2.putText(frame, f"Emotion: {dominant_emotion}", (20, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # --- Dashboard Draw karna ---
    draw_dashboard(frame, emotion_scores, frame_w, frame_h)

    cv2.imshow('Emotion Analysis Dashboard (Hugging Face)', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
print("Emotion detection (Hugging Face) stopped.")

c:\Users\ICAI\Desktop\coding_imp\ML_23052025\python\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Hugging Face model... (Yeh pehli baar time lega)


c:\Users\ICAI\Desktop\coding_imp\ML_23052025\python\.conda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ICAI\.cache\huggingface\hub\models--trpakov--vit-face-expression. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Error loading model: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.
Please check your internet connection or model name.
Webcam starting, Press 'q' to quit.
Error during analysis: name 'emotion_classifier' is not defined
Error during analysis: name 'emotion_classifier' is not defined
Error during analysis: name 'emotion_classifier' is not defined
Error during analysis: name 'emotion_classifier' is not defined
Error during analysis: name 'emotion_classifier' is not defined
Error during analysis: name 'emotion_classifier' is not defined
Error during analysis: name 'emotion_classifier' is not defined
Error during analysis: name 'emotion_classifier' is not defined
Error during analysis: name 'emotion_classifier' is not defined
Error during analysis: name 'emotion_classifier' is not defined
Error during analysis: name 'emotion_classifier' is not defined

: 